In [1]:
from aocd import get_data

puzzle_input = get_data(day=8, year=2023)

In [15]:
it1 = """
RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)
""".strip()

it2 = """
LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)
""".strip()

Partie 1

In [5]:
re.findall(r"[A-Z]+", "AAA = (BBB, CCC)")

['AAA', 'BBB', 'CCC']

In [19]:
import re

def parse_input(input: str):
    instructions, rows = input.split("\n\n")
    graph = {}
    for row in rows.split("\n"):
        nodes = re.findall(r"[A-Z0-9]+", row)
        for node in nodes:
            if node not in graph:
                graph[node] = {"L": None, "R": None}
            start, left, right = nodes
            graph[start]["L"] = left
            graph[start]["R"] = right

    return instructions, graph

it1_insts, it1_graph = parse_input(it1)
it1_insts, it1_graph 

('RL',
 {'AAA': {'L': 'BBB', 'R': 'CCC'},
  'BBB': {'L': 'DDD', 'R': 'EEE'},
  'CCC': {'L': 'ZZZ', 'R': 'GGG'},
  'DDD': {'L': 'DDD', 'R': 'DDD'},
  'EEE': {'L': 'EEE', 'R': 'EEE'},
  'ZZZ': {'L': 'ZZZ', 'R': 'ZZZ'},
  'GGG': {'L': 'GGG', 'R': 'GGG'}})

In [14]:
from itertools  import cycle

def follow_instruction(
    instructions: str,
    graph: dict,
    start_node: str = "AAA",
    end_node: str = "ZZZ",
) -> int:
    current_node = start_node
    instructions_iterator = cycle(instructions)
    nb_steps = 0

    while current_node != end_node:
        inst = next(instructions_iterator)
        current_node = graph[current_node][inst]
        nb_steps += 1

    return nb_steps

follow_instruction(*parse_input(it1))


2

In [16]:
follow_instruction(*parse_input(it2))

6

In [17]:
follow_instruction(*parse_input(puzzle_input))

21251

Partie 2

In [20]:
it3 = """
LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)
""".strip()

parse_input(it3)

('LR',
 {'11A': {'L': '11B', 'R': 'XXX'},
  '11B': {'L': 'XXX', 'R': '11Z'},
  'XXX': {'L': 'XXX', 'R': 'XXX'},
  '11Z': {'L': '11B', 'R': 'XXX'},
  '22A': {'L': '22B', 'R': 'XXX'},
  '22B': {'L': '22C', 'R': '22C'},
  '22C': {'L': '22Z', 'R': '22Z'},
  '22Z': {'L': '22B', 'R': '22B'}})

In [30]:
from itertools import cycle


def follow_instruction2(instructions: str, graph: dict) -> int:
    """Brute force"""
    
    instructions_iterator = cycle(instructions)
    nb_steps = 0

    current_nodes = [node for node in graph if node.endswith("A")]
    nb_nodes = len(current_nodes)

    for inst in instructions_iterator:

        stop = True
        for n in range(nb_nodes):
            current_nodes[n] = graph[current_nodes[n]][inst]

            if not current_nodes[n].endswith("Z"):
                stop = False
        
        nb_steps += 1
        if stop:
            break

    return nb_steps

follow_instruction2(*parse_input(it3))

6

In [31]:
follow_instruction2(*parse_input(puzzle_input))

KeyboardInterrupt: 

In [ ]:
from itertools import cycle


def follow_instruction2(instructions: str, graph: dict) -> dict:
    instructions_iterator = cycle(instructions)

    start_nodes = [node for node in graph if node.endswith("A")]
    
    nodes = {node: {"seen": {node}, "current": node, "cycle": 0} for node in start_nodes}
    remaining = set(start_nodes)
    keep = set()

    for inst in instructions_iterator:
        for start_node in remaining:
            current_node = nodes[start_node]["current"]
            current_node = graph[current_node][inst]
            nodes[start_node]["cycle"] += 1

            if not current_node in nodes[start_node]["seen"]:
                nodes[start_node]["seen"].add(start_node)

        remaining = keep
        keep = set()

        if not remaining:
            break

    return nodes


follow_instruction2(*parse_input(it3))